In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS mimic


In [0]:
d_cpt_df = spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/mnt/sravs/D_CPT.csv")

In [0]:
d_cpt_df.write.format("delta").mode("overwrite").saveAsTable("mimic.d_cpt")

In [0]:
query = """
WITH category_summary AS (
    SELECT 
        category,
        CASE 
            WHEN category = 1 THEN 'Traditional CPT Codes'
            WHEN category = 2 THEN 'Performance Measurement' 
            WHEN category = 3 THEN 'Emerging Technology'
            ELSE 'Other'
        END AS category_name,
        COUNT(*) as subsection_count,
        SUM(maxcodeinsubsection - mincodeinsubsection + 1) as total_code_range,
        ROUND(AVG(maxcodeinsubsection - mincodeinsubsection + 1), 2) as avg_codes_per_subsection
    FROM mimicdata.mimic.d_cpt
    GROUP BY category
)
SELECT 
    category_name,
    subsection_count,
    total_code_range,
    avg_codes_per_subsection,
    ROUND(100.0 * subsection_count / SUM(subsection_count) OVER(), 2) as pct_of_subsections,
    ROUND(100.0 * total_code_range / SUM(total_code_range) OVER(), 2) as pct_of_total_codes
FROM category_summary
ORDER BY category;
"""

df = spark.sql(query)
display(df)

category_name,subsection_count,total_code_range,avg_codes_per_subsection,pct_of_subsections,pct_of_total_codes
Traditional CPT Codes,123,80443,654.01,91.79,98.07
Performance Measurement,9,1391,154.56,6.72,1.70
Emerging Technology,2,196,98.0,1.49,0.24


In [0]:
%sql
SELECT 
    subsectionheader,
    sectionheader,
    subsectionrange,
    (maxcodeinsubsection - mincodeinsubsection + 1) as code_count,
    CASE 
        WHEN category = 1 THEN 'Traditional'
        WHEN category = 2 THEN 'Performance'
        WHEN category = 3 THEN 'Emerging Tech'
    END as category_type,
    RANK() OVER (ORDER BY (maxcodeinsubsection - mincodeinsubsection + 1) DESC) as size_rank
FROM mimicdata.mimic.d_cpt
ORDER BY code_count DESC
LIMIT 10;

subsectionheader,sectionheader,subsectionrange,code_count,category_type,size_rank
Musculoskeletal system,Surgery,20000-29999,10000,Traditional,1
Digestive system,Surgery,40490-49999,9510,Traditional,2
Integumentary system,Surgery,10040-19499,9460,Traditional,3
Diagnostic imaging,Radiology,70000-76499,6500,Traditional,4
Cardiovascular system,Surgery,33010-37799,4790,Traditional,5
Nervous system,Surgery,61000-64999,4000,Traditional,6
Urinary system,Surgery,50010-53899,3890,Traditional,7
Eye and ocular adnexa,Surgery,65091-68899,3809,Traditional,8
Respiratory system,Surgery,30000-32999,3000,Traditional,9
Chemistry,Pathology and laboratory,82000-84999,3000,Traditional,9


In [0]:
%sql
SELECT 
    sectionheader,
    COUNT(*) as subsection_count,
    SUM(maxcodeinsubsection - mincodeinsubsection + 1) as total_codes,
    MIN(maxcodeinsubsection - mincodeinsubsection + 1) as min_range_size,
    MAX(maxcodeinsubsection - mincodeinsubsection + 1) as max_range_size,
    ROUND(AVG(maxcodeinsubsection - mincodeinsubsection + 1), 2) as avg_range_size,
    ROUND(STDDEV(maxcodeinsubsection - mincodeinsubsection + 1), 2) as range_std_dev
FROM mimicdata.mimic.d_cpt
GROUP BY sectionheader
ORDER BY total_codes DESC;

sectionheader,subsection_count,total_codes,min_range_size,max_range_size,avg_range_size,range_std_dev
Surgery,19,57120,1,10000,3006.32,3329.78
Radiology,7,9588,9,6500,1369.71,2368.3
Pathology and laboratory,18,7101,3,3000,394.5,730.49
Medicine,32,5140,2,850,160.63,242.27
Performance measurement,9,1391,15,568,154.56,183.88
Anesthesia,22,1269,2,125,57.68,36.5
Evaluation and management,25,225,1,49,9.0,9.89
Emerging technology,2,196,4,192,98.0,132.94


In [0]:
%sql
WITH subsection_metrics AS (
    SELECT 
        row_id,
        sectionheader,
        subsectionheader,
        subsectionrange,
        category,
        (maxcodeinsubsection - mincodeinsubsection + 1) as code_count,
        mincodeinsubsection,
        maxcodeinsubsection,
        -- Calculate relative size within section
        ROUND(100.0 * (maxcodeinsubsection - mincodeinsubsection + 1) / 
              SUM(maxcodeinsubsection - mincodeinsubsection + 1) OVER (PARTITION BY sectionheader), 2) as pct_of_section,
        -- Calculate percentile ranking
        PERCENT_RANK() OVER (ORDER BY maxcodeinsubsection - mincodeinsubsection + 1) as size_percentile
    FROM mimicdata.mimic.d_cpt
)
SELECT 
    sectionheader,
    subsectionheader,
    subsectionrange,
    code_count,
    pct_of_section,
    ROUND(size_percentile * 100, 1) as size_percentile_rank,
    CASE 
        WHEN size_percentile >= 0.8 THEN 'Large'
        WHEN size_percentile >= 0.5 THEN 'Medium'
        WHEN size_percentile >= 0.2 THEN 'Small'
        ELSE 'Very Small'
    END as size_category,
    CASE 
        WHEN category = 1 THEN 'Traditional'
        WHEN category = 2 THEN 'Performance'
        WHEN category = 3 THEN 'Emerging Tech'
    END as category_type
FROM subsection_metrics
ORDER BY code_count DESC, sectionheader, subsectionheader;

sectionheader,subsectionheader,subsectionrange,code_count,pct_of_section,size_percentile_rank,size_category,category_type
Surgery,Musculoskeletal system,20000-29999,10000,17.51,100.0,Large,Traditional
Surgery,Digestive system,40490-49999,9510,16.65,99.2,Large,Traditional
Surgery,Integumentary system,10040-19499,9460,16.56,98.5,Large,Traditional
Radiology,Diagnostic imaging,70000-76499,6500,67.79,97.7,Large,Traditional
Surgery,Cardiovascular system,33010-37799,4790,8.39,97.0,Large,Traditional
Surgery,Nervous system,61000-64999,4000,7.00,96.2,Large,Traditional
Surgery,Urinary system,50010-53899,3890,6.81,95.5,Large,Traditional
Surgery,Eye and ocular adnexa,65091-68899,3809,6.67,94.7,Large,Traditional
Pathology and laboratory,Chemistry,82000-84999,3000,42.25,93.2,Large,Traditional
Surgery,Respiratory system,30000-32999,3000,5.25,93.2,Large,Traditional


In [0]:
%sql
WITH section_stats AS (
    SELECT 
        sectionheader,
        sectionrange,
        COUNT(*) as subsection_count,
        SUM(maxcodeinsubsection - mincodeinsubsection + 1) as total_codes,
        ROUND(AVG(maxcodeinsubsection - mincodeinsubsection + 1), 0) as avg_codes_per_subsection,
        -- Identify if section has deleted codes
        SUM(CASE WHEN LOWER(subsectionheader) LIKE '%deleted%' THEN 1 ELSE 0 END) as deleted_subsections,
        -- Count subsections by size
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 1000 THEN 1 ELSE 0 END) as large_subsections,
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) BETWEEN 100 AND 999 THEN 1 ELSE 0 END) as medium_subsections,
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) < 100 THEN 1 ELSE 0 END) as small_subsections
    FROM mimicdata.mimic.d_cpt
    GROUP BY sectionheader, sectionrange
)
SELECT 
    sectionheader,
    subsection_count,
    total_codes,
    avg_codes_per_subsection,
    deleted_subsections,
    large_subsections,
    medium_subsections,
    small_subsections,
    ROUND(100.0 * total_codes / SUM(total_codes) OVER(), 2) as pct_of_all_codes,
    ROUND(100.0 * deleted_subsections / subsection_count, 1) as pct_deleted_subsections
FROM section_stats
ORDER BY total_codes DESC;

sectionheader,subsection_count,total_codes,avg_codes_per_subsection,deleted_subsections,large_subsections,medium_subsections,small_subsections,pct_of_all_codes,pct_deleted_subsections
Surgery,19,57120,3006.0,2,10,5,4,69.63,10.5
Radiology,7,9588,1370.0,0,2,2,3,11.69,0.0
Pathology and laboratory,18,7101,395.0,0,1,10,7,8.66,0.0
Medicine,32,5140,161.0,1,0,14,18,6.27,3.1
Performance measurement,9,1391,155.0,0,0,3,6,1.70,0.0
Anesthesia,22,1269,58.0,1,0,2,20,1.55,4.5
Evaluation and management,25,225,9.0,4,0,0,25,0.27,16.0
Emerging technology,1,192,192.0,0,0,1,0,0.23,0.0
Emerging technology,1,4,4.0,0,0,0,1,0.00,0.0


In [0]:
%sql
SELECT 
    'Code Range Distribution' as analysis_type,
    COUNT(*) as total_subsections,
    COUNT(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) = 1 THEN 1 END) as single_code_subsections,
    COUNT(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) BETWEEN 2 AND 10 THEN 1 END) as small_range_subsections,
    COUNT(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) BETWEEN 11 AND 50 THEN 1 END) as medium_range_subsections,
    COUNT(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) BETWEEN 51 AND 200 THEN 1 END) as large_range_subsections,
    COUNT(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) > 200 THEN 1 END) as very_large_range_subsections
FROM mimicdata.mimic.d_cpt

UNION ALL

SELECT 
    'Category Breakdown' as analysis_type,
    COUNT(*) as total_subsections,
    COUNT(CASE WHEN category = 1 THEN 1 END) as traditional_codes,
    COUNT(CASE WHEN category = 2 THEN 1 END) as performance_codes,
    COUNT(CASE WHEN category = 3 THEN 1 END) as emerging_tech_codes,
    0 as large_range_subsections,
    0 as very_large_range_subsections
FROM mimicdata.mimic.d_cpt;

analysis_type,total_subsections,single_code_subsections,small_range_subsections,medium_range_subsections,large_range_subsections,very_large_range_subsections
Code Range Distribution,134,3,37,21,40,33
Category Breakdown,134,123,9,2,0,0


In [0]:
%sql
WITH section_stats AS (
    SELECT 
        sectionheader,
        sectionrange,
        COUNT(*) as subsection_count,
        SUM(maxcodeinsubsection - mincodeinsubsection + 1) as total_codes,
        ROUND(AVG(maxcodeinsubsection - mincodeinsubsection + 1), 0) as avg_codes_per_subsection,
        -- Identify if section has deleted codes
        SUM(CASE WHEN LOWER(subsectionheader) LIKE '%deleted%' THEN 1 ELSE 0 END) as deleted_subsections,
        -- Count subsections by size
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 1000 THEN 1 ELSE 0 END) as large_subsections,
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) BETWEEN 100 AND 999 THEN 1 ELSE 0 END) as medium_subsections,
        SUM(CASE WHEN (maxcodeinsubsection - mincodeinsubsection + 1) < 100 THEN 1 ELSE 0 END) as small_subsections
    FROM mimicdata.mimic.d_cpt
    GROUP BY sectionheader, sectionrange
)
SELECT 
    sectionheader,
    subsection_count,
    total_codes,
    avg_codes_per_subsection,
    deleted_subsections,
    large_subsections,
    medium_subsections,
    small_subsections,
    ROUND(100.0 * total_codes / SUM(total_codes) OVER(), 2) as pct_of_all_codes,
    ROUND(100.0 * deleted_subsections / subsection_count, 1) as pct_deleted_subsections
FROM section_stats
ORDER BY total_codes DESC;

sectionheader,subsection_count,total_codes,avg_codes_per_subsection,deleted_subsections,large_subsections,medium_subsections,small_subsections,pct_of_all_codes,pct_deleted_subsections
Surgery,19,57120,3006.0,2,10,5,4,69.63,10.5
Radiology,7,9588,1370.0,0,2,2,3,11.69,0.0
Pathology and laboratory,18,7101,395.0,0,1,10,7,8.66,0.0
Medicine,32,5140,161.0,1,0,14,18,6.27,3.1
Performance measurement,9,1391,155.0,0,0,3,6,1.70,0.0
Anesthesia,22,1269,58.0,1,0,2,20,1.55,4.5
Evaluation and management,25,225,9.0,4,0,0,25,0.27,16.0
Emerging technology,1,192,192.0,0,0,1,0,0.23,0.0
Emerging technology,1,4,4.0,0,0,0,1,0.00,0.0


In [0]:
%sql
SELECT 
    sectionheader,
    subsectionheader,
    subsectionrange,
    (maxcodeinsubsection - mincodeinsubsection + 1) as current_code_count,
    category,
    -- Market size indicator (based on code range)
    CASE 
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 1000 THEN 'Large Market'
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 100 THEN 'Medium Market'
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 10 THEN 'Small Market'
        ELSE 'Niche Market'
    END as market_size,
    -- Growth potential (emerging tech = high, deleted = none, etc.)
    CASE 
        WHEN category = 3 THEN 'High Growth Potential'
        WHEN LOWER(subsectionheader) LIKE '%deleted%' THEN 'No Growth (Discontinued)'
        WHEN category = 2 THEN 'Stable Growth'
        ELSE 'Moderate Growth Potential'
    END as growth_potential,
    -- Strategic priority scoring
    CASE 
        WHEN category = 3 THEN 5  -- Emerging tech highest priority
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 1000 AND category = 1 THEN 4
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 100 AND category = 1 THEN 3
        WHEN category = 2 THEN 2
        WHEN LOWER(subsectionheader) LIKE '%deleted%' THEN 0
        ELSE 1
    END as strategic_priority_score
FROM mimicdata.mimic.d_cpt
ORDER BY strategic_priority_score DESC, current_code_count DESC;

sectionheader,subsectionheader,subsectionrange,current_code_count,category,market_size,growth_potential,strategic_priority_score
Emerging technology,Temporary codes,0016T-0207T,192,3,Medium Market,High Growth Potential,5
Emerging technology,Temporary codes,0256T-0259T,4,3,Niche Market,High Growth Potential,5
Surgery,Musculoskeletal system,20000-29999,10000,1,Large Market,Moderate Growth Potential,4
Surgery,Digestive system,40490-49999,9510,1,Large Market,Moderate Growth Potential,4
Surgery,Integumentary system,10040-19499,9460,1,Large Market,Moderate Growth Potential,4
Radiology,Diagnostic imaging,70000-76499,6500,1,Large Market,Moderate Growth Potential,4
Surgery,Cardiovascular system,33010-37799,4790,1,Large Market,Moderate Growth Potential,4
Surgery,Nervous system,61000-64999,4000,1,Large Market,Moderate Growth Potential,4
Surgery,Urinary system,50010-53899,3890,1,Large Market,Moderate Growth Potential,4
Surgery,Eye and ocular adnexa,65091-68899,3809,1,Large Market,Moderate Growth Potential,4


In [0]:
%sql
SELECT 
    'Total Subsections' as metric,
    CAST(COUNT(*) AS VARCHAR(100)) as value,
    'All CPT subsections in dataset' as description
FROM mimicdata.mimic.d_cpt

UNION ALL

SELECT 
    'Total Code Range Coverage' as metric,
    CAST(SUM(maxcodeinsubsection - mincodeinsubsection + 1) AS VARCHAR(100)) as value,
    'Total individual codes covered' as description
FROM mimicdata.mimic.d_cpt

UNION ALL

SELECT 
    'Largest Section' as metric,
    sectionheader as value,
    'Section with most total codes' as description
FROM (
    SELECT sectionheader, SUM(maxcodeinsubsection - mincodeinsubsection + 1) as total_codes
    FROM mimicdata.mimic.d_cpt
    GROUP BY sectionheader 
    ORDER BY total_codes DESC 
    LIMIT 1
) largest

UNION ALL

SELECT 
    'Emerging Tech Procedures' as metric,
    CAST(COUNT(*) AS VARCHAR(100)) as value,
    'Future growth opportunities' as description
FROM mimicdata.mimic.d_cpt
WHERE category = 3

UNION ALL

SELECT 
    'Discontinued Procedures' as metric,
    CAST(COUNT(*) AS VARCHAR(100)) as value,
    'Subsections marked as deleted' as description
FROM mimicdata.mimic.d_cpt 
WHERE LOWER(subsectionheader) LIKE '%deleted%'

ORDER BY metric;

metric,value,description
Discontinued Procedures,8,Subsections marked as deleted
Emerging Tech Procedures,2,Future growth opportunities
Largest Section,Surgery,Section with most total codes
Total Code Range Coverage,82030,Total individual codes covered
Total Subsections,134,All CPT subsections in dataset


In [0]:
%sql
SELECT 
    row_id,
    category,
    sectionheader,
    subsectionheader,
    subsectionrange,
    mincodeinsubsection,
    maxcodeinsubsection,
    (maxcodeinsubsection - mincodeinsubsection + 1) as code_count,
    -- Create month/quarter placeholders for trend analysis
    'YYYY-MM' as period_placeholder,
    0 as volume_placeholder,
    0 as revenue_placeholder,
    -- Pre-calculate key metrics for joining with actual sales data
    CASE 
        WHEN category = 1 THEN 'Traditional'
        WHEN category = 2 THEN 'Performance'
        WHEN category = 3 THEN 'Emerging'
    END as category_name,
    CASE 
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 1000 THEN 'Large'
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 100 THEN 'Medium'
        WHEN (maxcodeinsubsection - mincodeinsubsection + 1) >= 10 THEN 'Small'
        ELSE 'Micro'
    END as size_segment
FROM mimicdata.mimic.d_cpt
ORDER BY category, sectionheader, mincodeinsubsection;


row_id,category,sectionheader,subsectionheader,subsectionrange,mincodeinsubsection,maxcodeinsubsection,code_count,period_placeholder,volume_placeholder,revenue_placeholder,category_name,size_segment
26,1,Anesthesia,Head,00100-00222,100,222,123,YYYY-MM,0,0,Traditional,Medium
27,1,Anesthesia,Neck,00300-00352,300,352,53,YYYY-MM,0,0,Traditional,Small
28,1,Anesthesia,Thorax,00400-00474,400,474,75,YYYY-MM,0,0,Traditional,Small
29,1,Anesthesia,Intrathoracic,00500-00580,500,580,81,YYYY-MM,0,0,Traditional,Small
30,1,Anesthesia,Spine and spinal cord,00600-00670,600,670,71,YYYY-MM,0,0,Traditional,Small
31,1,Anesthesia,Upper abdomen,00700-00797,700,797,98,YYYY-MM,0,0,Traditional,Small
32,1,Anesthesia,Lower abdomen,00800-00882,800,882,83,YYYY-MM,0,0,Traditional,Small
33,1,Anesthesia,Perineum,00902-00952,902,952,51,YYYY-MM,0,0,Traditional,Small
34,1,Anesthesia,Pelvis (except hip),01112-01190,1112,1190,79,YYYY-MM,0,0,Traditional,Small
35,1,Anesthesia,Upper leg (except knee),01200-01274,1200,1274,75,YYYY-MM,0,0,Traditional,Small
